<a href="https://colab.research.google.com/github/whykitsune/missing_spaces/blob/main/avito_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Задача** — разработать модель или алгоритм, который принимает на вход текст без пробелов и возвращает восстановленный текст с правильными пробелами и позициями, где они были пропущены. Для решения дообучим модель с Hugging Face, а именно *bert-base-multilingual-cased*

Почему выбран именно такой подход?

Чтобы добиться высокого качества, модель должна быть предобучена и иметь какое-то представление о словах, поэтому был выбран вариант дообучить модель с HF. Обучать в данном случае нужно BERT-подобную модель, потому что модель, например, T5 будет генерировать новую последовательность, что может сильно исказить итоговый вариант строки, нам нужно ли классифицировать, в каких местах нужен пробел.

Обучение производилось на T4 GPU в Google Colab

### Импорт библиотек

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertModel, BertConfig
import pandas as pd
import numpy as np
from tqdm import tqdm

Установим тренировочный и тестовый датасеты

In [2]:
!gdown -q 1FkHimRiYDZjjh7tg5eFnsxnS9ELVVbTM

In [3]:
!gdown -q 1diH-nG6yMFR5HrM9xj0SXcebW-JYkpL9

### Подготовка данных

Приведем предоставленный csv-файл к pandas.DataFrame. Поскольку в данных внутри строчек объявлений бывают запятые, обработаем этот случай

In [4]:
data = []
with open("/content/dataset_1937770_3.txt", "r") as f:
    for line in f.readlines():
        line_split = line.split(",")
        data.append([line_split[0], ",".join(line_split[1:])[:-1]])

task_data = pd.DataFrame(data[1:], columns=data[0])
task_data.head()


,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку


Получим список строк предоставленного набора данных

In [5]:
task_texts = task_data["text_no_spaces"].to_list()

Возьмем для обучения [датасет объявлений с Авито](https://www.kaggle.com/datasets/vitaliy3000/avito-dataset?resource=download)

In [6]:
!unzip archive.zip -d archive

Archive:  archive.zip
  inflating: archive/category.csv    
  inflating: archive/test.csv        
  inflating: archive/train.csv       


In [7]:
train_data = pd.read_csv("/content/archive/train.csv", usecols=["item_id", "title"]).set_index("item_id")

# Половину строк приведем к lowercase'у (чтобы модель видела разные данные: и с регистром, и без)
indices = train_data.index.to_numpy()
np.random.shuffle(indices)
half_idx = indices[:len(indices)//2]
train_data.loc[half_idx, "title"] = train_data.loc[half_idx, "title"].str.lower()

In [8]:
train_data.head()

,title
item_id,
432777,картина
428289,стулья из прессованной кожи
35454,домашняя мини баня
290758,"эксклюзивная коллекция книг ""трансаэро"" + подарок"
398784,ноутбук aser


Получим списки строк тренировочных текстов (с пробелами и без)

In [21]:
train_texts_spaced = train_data["title"].to_list()[:70_000]
train_texts_no_space = [s.replace(" ", "") for s in train_texts_spaced]

Генерируем метки: 1 = пробел перед символом

In [22]:
labels = []
for s, ns in zip(train_texts_spaced, train_texts_no_space):
    lbl = []
    j = 0
    for i, ch in enumerate(ns):
        if i == 0:
            lbl.append(0)  # в начале никогда не ставим
        else:
            if s[j] == " ":
                lbl.append(1)
                j += 1
            else:
                lbl.append(0)
        j += 1
    labels.append(lbl)

Определим девайс для обучения

In [11]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Функции для обучения

In [23]:
class SpaceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Токенизация по символам (разбиваем текст вручную)
        chars = list(text)
        enc = self.tokenizer(
            chars,
            is_split_into_words=True,  # важно, чтобы символы шли по отдельности
            truncation=True,
            max_length=self.max_len,
            padding="max_length",
            return_tensors="pt"
        )

        # Метки на каждый символ (0/1 = пробел перед этим символом)
        label = label[:self.max_len] + [0] * (self.max_len - len(label))
        label = torch.tensor(label)

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": label
        }

In [24]:
class BertForSpaceRestoration(nn.Module):
    def __init__(self, model_name="bert-base-multilingual-cased", hidden_dropout_prob=0.1):
        super().__init__()
        config = BertConfig.from_pretrained(model_name, output_hidden_states=False)
        self.bert = BertModel.from_pretrained(model_name, config=config)
        self.dropout = nn.Dropout(hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 2)  # бинарная классификация (0/1)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        x = self.dropout(outputs.last_hidden_state)
        logits = self.classifier(x)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # labels: (batch, seq_len), logits: (batch, seq_len, 2)
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        return {"loss": loss, "logits": logits}

In [25]:
def train_model(train_loader, model, epochs=3, lr=5e-5, device=DEVICE):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    model.to(device)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask, labels)
            loss = outputs["loss"]
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss = {total_loss/len(train_loader):.4f}")

    return model

In [26]:
def predict_spaces(model, tokenizer, text, device=DEVICE):
    model.eval()
    chars = list(text)
    enc = tokenizer(chars, is_split_into_words=True, return_tensors="pt", truncation=True, max_length=128, padding="max_length").to(device)
    with torch.no_grad():
        logits = model(enc["input_ids"], enc["attention_mask"])["logits"]
    preds = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

    # Восстановим строку
    out = []
    for i, ch in enumerate(chars[:128]):  # ограничение длины
        if preds[i] == 1:  # пробел перед символом
            out.append(" ")
        out.append(ch)
    return "".join(out)

### Обучение

In [27]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

dataset = SpaceDataset(train_texts_no_space, labels, tokenizer)
loader = DataLoader(dataset, batch_size=8, shuffle=True)

model = BertForSpaceRestoration()

In [28]:
model = train_model(loader, model, epochs=3, device=DEVICE)

test_text = "книгавхорошемсостоянии"
restored = predict_spaces(model, tokenizer, test_text)
print("Input:   ", test_text)
print("Restored:", restored)

100%|██████████| 8750/8750 [33:20<00:00,  4.37it/s]


Epoch 1, Loss = 0.0165


100%|██████████| 8750/8750 [33:24<00:00,  4.37it/s]


Epoch 2, Loss = 0.0110


100%|██████████| 8750/8750 [33:22<00:00,  4.37it/s]

Epoch 3, Loss = 0.0092
Input:    книгавхорошемсостоянии
Restored: книга в хорошем состоянии


### Предсказания на тестовых данных

Получим список получившийхся строк (после вставления пробела моделью)

In [29]:
res_str = []
for txt in task_texts:
    res_str.append(predict_spaces(model, tokenizer, txt))

Напишем функцию для подготовки ответа в требуемом формате

In [30]:
def find_space_indices(text_with_spaces):
    text_without_spaces = text_with_spaces.replace(' ', '')
    result = []
    offset = 0

    for i, char in enumerate(text_with_spaces):
        if char == ' ':
            result.append(str(i - offset))
            offset += 1

    return result

Получим итоговые предсказания и сформируем csv

In [31]:
pred_positions = []
for line in res_str:
    cur_res = find_space_indices(line)
    pred_positions.append("[" + ", ".join(cur_res) + "]")

In [32]:
submission = task_data.copy()
submission = submission.assign(predicted_positions=pred_positions).drop(columns=["text_no_spaces"])
submission.to_csv('submission.csv')